In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap, cumsum
from bokeh.layouts import row, column, gridplot
from bokeh.models import DatetimeTickFormatter
from bokeh.models import NumeralTickFormatter


In [2]:
#load data
process_dataset = pd.read_csv('archive/supplier_stock_value.csv')
process_dataset['Date'] = pd.to_datetime(process_dataset['Date'],format='%d-%m-%Y')
process_dataset = process_dataset.sort_values('Date',ascending=True) 

##### plot by item
#link between pd and bokeh for original data
src1 = ColumnDataSource(process_dataset)


# create CDS for empty dataframe
filteredSource = ColumnDataSource(data=dict(Date=[],Stock_value=[]))


p = figure(title="Invetory level",
                x_axis_type="datetime",
                x_axis_label='Date',
                y_axis_label='Stock_value')


item_draw =p.line(x='Date',y='Stock_value',source = filteredSource,legend_label="Stock level",hover_color = 'red')

#format y axis
p.left[0].formatter.use_scientific = False
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,000")

#format x axis
p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d/%m/%Y"],
    )


#add hovertool
p.add_tools(HoverTool(
    tooltips=[
        ( "Date","$x{%F}"),
        ( "Stock_value", "$y{"+f"0.00"+" a}" )
    ],
    formatters={
        '$x' : 'datetime',
    },
))

p.legend.location = 'bottom_right'

vendor_list = sorted(process_dataset['Vendor'].unique().tolist())

#get list of item for box
item_list= ['Please choose item...'] + vendor_list


select = Select(title='Vendors', value=item_list[0], options=item_list)

callback = CustomJS(
    args=dict(src1=src1,filteredSource=filteredSource,select=select),
    code= """
    const data = src1.data;
    const item_filtered = select.value;
    const plotdata = filteredSource.data;
    plotdata['Stock_value']=[];
    plotdata['Date']=[];
    
    for (var i=0; i<data['Vendor'].length; i++) {
        if(data['Vendor'][i] == item_filtered ) {
            plotdata['Stock_value'].push(data['Stock_value'][i])
            plotdata['Date'].push(data['Date'][i])
        }
    }
    filteredSource.change.emit();
    """
)
select.js_on_change('value',callback)


layout = row(p,column(select))
show(layout)

In [3]:
process_dataset

,Vendor,Date,Stock_value
0,0 - 0,2022-06-15,1.565400e+06
572,60063 - Konnect,2022-06-15,8.201404e+02
569,60055 - SATO AUSTRALIA PTY LTD,2022-06-15,2.402227e+03
560,60022 - CMTP Pty Ltd,2022-06-15,4.904400e+03
557,60013 - Amcor Fibre Packaging,2022-06-15,0.000000e+00
...,...,...,...
635,60404 - ESCATEC ELECTRONICS SDN BHD,2022-06-29,0.000000e+00
810,71647 - Trafigura PTE Ltd,2022-06-29,2.032316e+05
412,37686 - Goldstar Co.Ltd,2022-06-29,0.000000e+00
406,37388 - Tritech Technology AB,2022-06-29,0.000000e+00
